<h1 align="center">Lab 12: Introduction to basic statistical analysis using the Allen Brain Connectivity Atlas</h1>
<p><center>This notebook will introduce you to the Allen Mouse Brain Connectivity Atlas tools and data and will guide you in conducting basic statistical analysis.</center></p>
<img src='./images/connectivity_website.png' width="500px"/>

<h3 align="center">Estimated Duration: 30 mins</h3>


### Table of Contents:


[Introduction](#section_intro)
 
**Part I: Brain Connectivity Statistics**
1. [Acquiring the Data](#section_data)
2. [Most Projected Structure](#section_projection)
3. [Projection Density Distribution](#section_distr)
4. [Medial Mammilary Nucleus: Average Projection Density](#section_mean)
5. [Average Projection Densities across all Structures](#section_avg_proj_density) 

**Part II: Bootstrap**
1. [Aquiring the Data](#section_data2)
2. [Medial Mammilary Nucleus DataFrame](#section_mmn)
3. [Bootstrapping](#section_bootstrapping)
4. [95% Confidence Interval](#section_ci)
5. [The P-Value](#section_pval)



# Introduction<a id='section_intro'></a>

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
The <b>Allen Brain Institute for Brain Science</b> is a non-profit research institute that utilizes big science to understand specific aspects of brain function. The insitute has many publicly available atlases online that provide information on gene expression and neural connectivity. The atlas that we will be working with today is the Mouse Brain Connectivity Atlas, defined as "a brain-wide map of neural projections, including cell class specific data." 

In this part of the lab, we will utilize information from the Mouse Brain Connectivity Atlas to make conclusions regarding structural connectivity and to further explore projection information.
</div>

In [2]:
!pip install -r requirements


  ERROR: Could not find a version that satisfies the requirement allensdk.core.mouse_connectivity_cache (from -r requirements (line 9)) (from versions: none)
ERROR: No matching distribution found for allensdk.core.mouse_connectivity_cache (from -r requirements (line 9))


In [ ]:
!pip install -r requirements

In [ ]:
# Run this cell
!pip install -r requirements

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from allensdk.api.queries.ontologies_api import OntologiesApi

%matplotlib inline 
import seaborn as sns

# Activates the API and cache
mcc = MouseConnectivityCache()
structure_tree = mcc.get_structure_tree()


<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
On this lab, we will obtain specific information about individual structures of the brain and the experiments done on them, information that we will later be able to use to conduct basic statistical analyses and make conclusions regarding connectivity. 

We will start by running the cell below, which sets up all the variables that will be needed throughout the lab.


</div>

# Part 1: Brain Connectivity Statistics


## Part 1.1: Aquiring the Data<a id='section_data'></a>

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">

Running the cell below, you will obtain the data that we will be using in this lab. This data contains a set of experiments done on the Subiculum (Structure ID: 502) in wild-type mice. You don't need to worry about what these variables mean, but if you would like to learn more, you can go to <a href="https://allensdk.readthedocs.io/en/latest/_static/examples/nb/mouse_connectivity.html">Mouse Connectivity</a> for reference.

</div>

In [ ]:
summary_structures = structure_tree.get_structures_by_set_id([167587189])
summary_structure_ids = [item['id'] for item in summary_structures]

experiments = mcc.get_experiments(injection_structure_ids=[502]) 
experiment_set = [exp['id'] for exp in experiments if exp['transgenic_line'] == None]

name_map = structure_tree.get_name_map()

unionizes_set = mcc.get_structure_unionizes(experiment_set,
                                               is_injection = False,
                                                structure_ids = summary_structure_ids,
                                               hemisphere_ids = [3])

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
We have now gathered all the information needed to do statistical and data analysis. Our `unionizes_set` DataFrame (fancy word for a table) consists of a list of structures where the Subiculum projects to when it is injected. Some of the columns in this DataFrame that we will be analyzing are:

</div>

- `projection_density`: The projection density is defined as the sum of detected pixels in the image/ sum of all pixels in division. It represents the strength of the connection between two structures in the brain as the more colored pixels there are in the image, the stronger the connection.
- `structure_id`: A unique number assigned to every individual structure in the brain by the Allen Insitute for Brain Science.
    


## Part 1.2: Most Projected Structure<a id='section_projection'></a>

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
Let's check to see which structure has the largest projection density. 
    
To do this, we want use the <code>sort_values</code> function with the correct column label (projection density) and order (descending). This will output a DataFrame with projection densities sorted from largest to smallest. Run the cell below and  find the <code>structure_id</code> of the first row. Assign this number to <code>mystery_id</code> in the next cell.
</div>

In [ ]:
index = unionizes_set.sort_values('projection_density', ascending=False)[['hemisphere_id', 'id', 'projection_density', 'projection_volume', 'experiment_id', 'structure_id']]
index.head()

In [ ]:
mystery_id = ...

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
We can't immediately recognize what structure corresponds to this id. The function <code>name_map</code> provides us with the name of the structure that corresponds to the structure id. Run the cell below to find what the structure is!
</div>

In [ ]:
# Run this cell
name_map[mystery_id]

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
Does our result make sense? Write down in the cell below why or why not. 

<p><b>Hint:</b>Where should the subiculum project to?</p>
</div>

*Your Answer Here*

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
Now that we know the structure id and name of where the highest projection density is, let's get a subset of that data. Run the cell below to get a DataFrame of structure ids that are equal to number we found in the previous question.
</div>

In [ ]:
# Run this cell 
medial_mammillary = unionizes_set[unionizes_set['structure_id'] == mystery_id]
medial_mammillary


## Part 1.3: Projection Density Distribution<a id='section_distr'></a>

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
Let's observe the distribution of values in the Medial Mammilary Nucleus. Recall in statistcs the distribution of values in a normal distribution:


<img src='./images/normal.png' width="400px"/>
<i>Image from Illinois State University</i>

<p>First, set <code>proj_dens</code> to a list of projection density values using <code>.values</code>. Then, run the cell to visualize the distribution of projection densities in the medial mammilary nucleus.</p>
</div>

In [ ]:
# Gives us a list of projection_densities in the medial mammilary nucleus
proj_dens = medial_mammillary['projection_density'].values


plt.title('Distribution of Projection Densities at the Medial Mammilary Nucleus')
plt.xlabel("Projection Density")
sns.kdeplot(proj_dens);

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">

Based on the distribution above, what do you think the values of the mean, median, and standard deviation are? Type your answer in the cell below:
</div>

*Your Answer Here*


## Part 1.4: Medial Mammilary Nucleus: Average Projection Density<a id='section_mean'></a>

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">

Let's calculate the <b>mean of the projection densities</b> from the subicculum to <b>just</b> the medial mammillary nucleus. In other words, of all the projections from the subicculum to the medial mammillary nucleus, what is the projection density on average? Assign this value to <code>mean</code>.

<p><b>Hint:</b> Recall the functions <code>sum</code> and <code>len</code> that allow you to calculate the mean as discussed in the Pre-Lab.</p>
</div>

In [ ]:
sum_values = ...
num_values = ...
mean = ... / ...
mean


## Part 1.5: Average Projection Densities across all Structures<a id='section_avg_proj_density'></a>

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
Using our statistical analysis, we can observe the average projection densities at the top 20 sites where the subiculum maps to for the 5 wildtype injections. In other words, for each of the 20 structures that the subiculum maps to, we will calculate the average projection density. Luckily, the Connectivity API provides an easy way to display these averages: <b>Simply run the cell below to visualize a bar chart representation of this.</b>
</div>

In [ ]:
# Run this cell

# We can combine groupby and mean to find the average projection density across experiments for each summary structure
unionizes_set_mean = (unionizes_set.groupby('structure_id', as_index = False)['projection_density'].mean())

# Again, add a column with summary structure acronyms so we can interpret the unionizes more easily
names = [name_map[strid] for strid in unionizes_set_mean['structure_id']]
unionizes_set_mean['structure_name'] = names
unionizes_set_mean.sort_values(by = 'projection_density', ascending = False, inplace = True)

# Plot the 20 structures with the highest average projection density across all wild type Subiculum injections
unionizes_set_mean[:20].plot('structure_name', 'projection_density', kind='barh')
plt.gca().invert_yaxis()
plt.title('Average Unionize Data for %i injections'%len(experiment_set));

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
The cell below provides statistics (i.e. mean and standard deviation) of the projection densities from the subiculum to various structures.
</div>

In [ ]:
# look at the data
# re-generate means so they are in the same order as standard deviation
unionizes_set_mean = (
    unionizes_set.groupby('structure_id', as_index = False)[
    'projection_density'].mean())

# generate a second data frame with standard deviations
unionizes_set_std = (
    unionizes_set.groupby('structure_id', as_index = False)[
    'projection_density'].std())

# find names from structure ids
names = [name_map[strid] for strid in unionizes_set_mean['structure_id']]

# combine names, mean, and standard deviation into one dataframe for easy plotting
set_mean_std = pd.DataFrame({'structure_name': names, 
                            'mean_projection_density': unionizes_set_mean['projection_density'], 
                            'stdev':  unionizes_set_std['projection_density']
                            })

# sort the dataframe to get the top 20 values on top
set_mean_std.sort_values(by = 'mean_projection_density', ascending = False, inplace = True)

set_mean_std.head()

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
Do your results match those in our visualization and the table above? Explain in the cell below.
</div>

*Your Answer Here.*

# Part 2: Bootstrap


## Part 2.1: Aquiring the Data<a id='section_data2'></a>

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">

For this part of the lab, we will apply a technique called boostraping to analyze the probability of a structure being a part of a circuit. We will start off with looking at the Primary Visual Cortex since it has more wildtype injections than the subiculum, with 33 wildtype samples. 

The <code>visual_unionizes_set</code> consists of all of the structures that the Primary visual area projects to for every experiment done.
</div>

In [ ]:
# Run this cell
VISp = structure_tree.get_structures_by_name(["Primary visual area"])
VISp_id = VISp[0]['id']
visual_experiments = mcc.get_experiments(injection_structure_ids=[VISp_id]) 
visual_experiment_set = [exp['id'] for exp in visual_experiments if exp['transgenic_line'] == None]
visual_data = pd.DataFrame(visual_experiment_set)

visual_unionizes_set = mcc.get_structure_unionizes(visual_experiment_set,
                                               is_injection = False,
                                                structure_ids = summary_structure_ids,
                                               hemisphere_ids = [3])
visual_names = [name_map[strid] for strid in visual_unionizes_set['structure_id']]
visual_unionizes_set['structure_name'] = visual_names

In [ ]:
# Run this cell
visual_unionizes_set.head()


## Part 2.2: Medial Mammilary Nucleus DataFrame<a id='section_mmn'></a>

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">

In the next cell, we want to obtain the projection densities for the different injections to the Primary visual area that project to the medial mammillary nucleus. We do this in the first line and we name it <code>mmn</code> to differentiate from the variable <code>medial_mammillary</code>. Next, we arrange the projection densities from biggest to smallest. We will place these projection densities in a DataFrame that we can then use for boostrapping. 
</div>

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
In the next cell, use the <code>sort_values</code> function to sort the values for <code>projection_density</code> from greatest to least and assign this table to <code>mmn_sorted</code>.
</div>

In [ ]:
mmn = visual_unionizes_set[visual_unionizes_set['structure_name'] == 'Medial mammillary nucleus']
mmn_sorted = ...
mmn_sorted

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
In the next cell, use your knowledge of the <code>.values</code> function and the <code>[...]</code> method to obtain an list with all the values for the projection densities. Set this equal to <code>mmn_values</code>.
</div>

In [ ]:
mmn_values = ...
mmn_values

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
Run the cells below to create a DataFrame with the <code>mmn_values</code>. We will use this table for bootstrapping. 
</div>

In [ ]:
# Run this cell
mmn_data = {'Medial mammillary nucleus': mmn_values}
mmn_dataframe = pd.DataFrame(data=mmn_data)
mmn_dataframe.head()


## Part 2.3: Bootstrapping<a id='section_bootstrapping'></a>

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
The essence of bootstrapping is reliant on a population. In this case, the population is going to be that of the neurons that project to the medial mammillary nucleus from the primary visual area. We obtained a sample, the 33 wildtype experiments from the primary visual area. 

Fill in the code below that will sample with replacement from the population 5000 times using the mean as our test statistic. This will make sure that our number is big enough to be accurate and representative of the population.

</div>

In [ ]:
# Run this cell
means = []
# We want a lot of resamples, so we resample 5000 times
for i in np.arange(5000):
    resampled = mmn_dataframe.sample(n = 33, replace = True)
    # We had 33 wildtype experiments, so n = 33. We set replace = True
    boot_mean = resampled.mean()
    means = np.append(means, boot_mean)
# We end with an array (list) of all the means of the 5000 resamples
means

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
We will now place the resampled means into a DataFrame and graph them into the histogram below. 
</div>

In [ ]:
# Run this cell
mmn = {'Projection Density Means': means}
mmn_means = pd.DataFrame(data=mmn).hist()

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
Now, let's overlay the mean projection density from the subiculum to the medial mammillary nucleus onto the histogram shown above.
</div>

In [ ]:
# Run this cell
mmn = {'Projection Density Means': means}
mmn_means = pd.DataFrame(data=mmn).hist()
plt.axvline(mean, color = 'k');

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
What can we see about the difference between the mean projection densities from primary visual area to the medial mammillary nucleus and the mean projection density from the subiculum to the medial mammillary nucleus?
</div>

*Your Answer Here.*


### What does this mean in terms of connectivity?

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
  
Since the subiculum and the medial mammillary nucleus are a part of the Papez Circuit, it makes sense why the mean projection density is large. In contrast, the mean projection density for the primary visual cortex to the medial mammillary nucleus is low, making this structure not a part of the Papez Circuit (even though there is still minimal connectivity).
</div>


## Part 2.4: 95% Confidence Interval<a id='section_ci'></a>

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">

Using the bootstrapped sample means calculated above, we will create a 95% confidence interval using the <code>quantile</code> function. This will give us an interval with the true mean of all projection densities from the primary visual area to the medial mammillary nucleus. 

Run the cell below to obtain the 95% Confidence Interval. Run the cell below that to overlay the 95% Confidence Interval over the projection density mean distribution.

</div>

In [ ]:
# Run this cell
mmn = {'Projection Density Means': means}
mmn_means = pd.DataFrame(data=mmn)
percentile = list(mmn_means.quantile([0.025, 0.975])['Projection Density Means'])
percentile

In [ ]:
# Run this cell
mmn_means.hist()
plt.hlines(y=0, xmin=percentile[0], xmax=percentile[1], linewidth=10, color = 'y');


## Part 2.5: The P-Value<a id='section_pval'></a>

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
Below, we will calculate the <b>p-value</b> using the above distribution and a random mean named <code>mystery_mean</code> that will simulate a mean projection density obtained from one of the injections into the primary visual area that projects to the medial mammillary nucleus. We must come up with a null hypothesis and an alternative hypothesis in order to use the p-value.

Write down your <b>Null Hypothesis</b> and your <b>Alternative Hypothesis</b> below. Run the cells and find the p-value for further analysis. 

</div>

**Null hypothesis**: *Your Answer Here.* 


**Alternative hypothesis**: *Your Answer Here.*

In [ ]:
# Run this cell
mystery_mean = 0.0434
mmn = {'Projection Density Means': means}
mmn_means = pd.DataFrame(data=mmn)

In [ ]:
# Run this cell
mmn_means.hist()
plt.axvline(mystery_mean, color = 'k');

In [ ]:
# Run this cell
p_value = np.average(mmn_means > mystery_mean)
p_value

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
Based on the p-value calculated above, do we reject or fail to reject the null hypothesis? Explain.
</div>

*Your Answer Here.*

# Conclusion

<div style="border-left: 3px solid #003262; padding: 1px; padding-left: 10px; background: #ffffff; ">
    
We hoped that this lab served as a good introductions to the tools and the statistical analysis used in neuroinformatics. Neuroinformatics is a growing field and every year, new techniques are being invented for collection of large neural information. Because of this, it is important to know how to analyze and manipulate this data. 

If you liked the lab and would like to further your data science techniques, below are some introductary courses that may interest you:

<p><i><b>Data 8: The Foundations of Data Science</i></b></p>

<p><i><b>Data 100: Principles and Techniques of Data Science</i></b></p>

<p><i><b>Prob 140: Probability for Data Science</i></b></p>
</div>


<i>Notebook Developed by: Elias Saravia & Daniel Lopez</i>